# Tracers and barriers paper

Tracers and Barriers paper

This script is for the generation of flow model input files, running the flow model, and the generation of transport model input files & running the transport model.

THEN --> post-process the data using the post-processing (tracers_barriers_PP.py) script.

## Case Study (CS) & recharge descriptions

CS1 is with no barrier.
CS2 is fully-penetrating barrier.
CS3 is buried barrier.

Recharge 1 = Diffuse recharge across the whole aquifer.
Recharge 2 = Upgradient (mountain-front) recharge only.

## Importing packages

In [34]:
import numpy as np
import matplotlib as mpl
import flopy
from flopy import mt3d
import pandas as pd
import subprocess
import os
import time
import sys
import datetime
from decimal import Decimal

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('pandas version: {}'.format(pd.__version__))
print('flopy version: {}'.format(flopy.__version__))

3.7.4 (default, Aug  9 2019, 18:34:13) [MSC v.1915 64 bit (AMD64)]
numpy version: 1.16.5
matplotlib version: 3.2.2
pandas version: 0.25.1
flopy version: 3.3.4


## Setting up directories

In [35]:
modelName           = "TB_01"
modelName_mt3d      = "%s_mt3d"%modelName

In [36]:
print("Flow model name is: %s, transport model name is: %s" %(modelName, modelName_mt3d))

Flow model name is: TB_01, transport model name is: TB_01_mt3d


In [37]:
MODFLOW_folder      = 'MODFLOW-NWT_64.exe'

project_folder      = os.getcwd()
MT3D_USGS_folder    = 'mt3d-usgs_1.1.0_64.exe'
mt3d_version        = 'mt3d-usgs' 

print("Project folder is: %s" %project_folder)

Project folder is: C:\Users\mar886\WaterTableProject\TracersBarriers\tracers_barriers


In [38]:
this_model_folder = project_folder  # Can make a new subfolder to save model output elsewhere
if not os.path.exists(this_model_folder):
    os.makedirs(this_model_folder)
    
print("This model folder is: %s" %this_model_folder)

dataDirectory = os.path.join(this_model_folder, "Data")
if not os.path.exists(dataDirectory):
    os.makedirs(dataDirectory)
    
print("The data directory is: %s" %dataDirectory)

figureDirectory = os.path.join(this_model_folder, "Figures")
if not os.path.exists(figureDirectory):
    os.makedirs(figureDirectory)

print("Directory to save figures: %s" %figureDirectory)

This model folder is: C:\Users\mar886\WaterTableProject\TracersBarriers\tracers_barriers
The data directory is: C:\Users\mar886\WaterTableProject\TracersBarriers\tracers_barriers\Data
Directory to save figures: C:\Users\mar886\WaterTableProject\TracersBarriers\tracers_barriers\Figures


## Model discretisation

In [39]:
nlay = 12                    # Number of layers.
Lx = 10000.                  # Length of the model sides in metres.
Ly = 5000                    # Rectangular domain.
delr = 25.                   # Spacing length across rows in metres.
delc = delr                  # The spacing along columns and rows are equal.
ncol = int(Lx/delr)          # Number of columns.
nrow = int(Ly/delc)          # Number of rows.
ncells = nlay*ncol*nrow      # Total number of cells in the model.
ncells_per_layer = ncol*nrow # Number of cells per layer.
surface_area = Lx*Ly         # m^2

ztop = 300.                  # Top of model.
zbot = 0.                    # Base of model.
delv = (ztop - zbot) / nlay  # Length of cells in z direction.
botm = np.linspace(ztop, zbot, nlay + 1) 

## Hydraulic properties

In [40]:
sHead = 290                  # Starting head across aquifer.

sy = 0.1                     # Specific yield.
ss = sy/ztop                 # Specific storage.
laytyp = 1                   # Layer type (1 = convertible; 0 = confined).

hk_aquifer = 1.              # Hydraulic conductvity along rows (m/day).
vka = hk_aquifer/10.         # Vertical hydraulic conductivity.

prsity = sy
tortuosity_factor = 0.09

recharge_flux = 1.3699e-5    # m/d This is equivalent to 5 mm per year.
total_recharge = recharge_flux*surface_area # m^3 per day.
total_recharge_per_y = total_recharge*365.25 # m^3 per year.

length_simulation = 1        # Length of the steady-state flow model (days). 

## Barrier characteristics

In [41]:
hk_barrier           = hk_aquifer/1000 # m/d 3 orders of magnitude lower than the aquifer.
barrier_width        = 50 # m
number_barrier_cols  = barrier_width/delr  # So we can test/change the grid spacing.
column_of_barrier    = []

if number_barrier_cols == 1:
    column_of_barrier.append(int(ncol/2)) 
elif number_barrier_cols%2 == 0: # Even number
    for i in range(int(number_barrier_cols/2)):
        a = int(number_barrier_cols/2) - i
        column_of_barrier.append(int(ncol/2)-a)

    for i in range(int(number_barrier_cols/2)):
        print(i)
        column_of_barrier.append(int(ncol/2)+i)
else: # Odd number        
    for i in range(int(number_barrier_cols/2)):
        a = int(number_barrier_cols/2) - i
        column_of_barrier.append(int(ncol/2)-a)

    for i in range(int(number_barrier_cols/2)):
        print(i)
        column_of_barrier.append(int(ncol/2)+i)

    column_of_barrier.append(int(ncol/2)+int(number_barrier_cols/2))
    
percentage_gap        = 10 # CS2 
first_row_barr        = int(nrow/percentage_gap) # CS2 
last_row_barr         = int(nrow-(nrow/percentage_gap)) # CS2 
start_lay_barr        = int(nlay/2) # CS3
end_lay_barr          = nlay # CS3

0


## Making the hydraulic conductivity arrays

In [42]:
number_of_casestudies = 3

Case Study 1: No barrier

In [43]:
hk_array_1 = hk_aquifer * np.ones((nlay, nrow, ncol), dtype=np.float32)      

Case Study 2: Fully-penetrating barrier

In [44]:
hk_array_2 = hk_aquifer * np.ones((nlay, nrow, ncol), dtype=np.float32)     
  
for il in range(nlay):
    for ir in range(first_row_barr, last_row_barr, 1): 
        for col_number in range(len(column_of_barrier)):
            hk_array_2[il, ir, column_of_barrier[col_number]] = hk_barrier 

Case Study 3: Buried barrier

In [45]:
hk_array_3 = hk_aquifer * np.ones((nlay, nrow, ncol), dtype=np.float32)     

for il in range(start_lay_barr, end_lay_barr):
    for ir in range(nrow): 
        for col_number in range(len(column_of_barrier)):
            hk_array_3[il, ir, column_of_barrier[col_number]] = hk_barrier 

## Recharge characteristics

In [46]:
number_of_recharge_scenarios = 2

Recharge array 1 = diffuse recharge across the whole aquifer in equal amounts.

In [47]:
recharge_array_1 = recharge_flux * np.ones((nrow, ncol), dtype=np.float32)

Recharge array 2 = recharge across only the top of the catchment.
Recharge across all rows, across first 50 columns (One quarter of catchment).

In [48]:
n_cols_rech_2    = int(ncol/4)
n_cells_rech_2   = int(n_cols_rech_2*nrow)
SA_recharge_2    = Ly*n_cols_rech_2*delc # Surface area (m^2)
recharge_flux_2  = total_recharge/SA_recharge_2 # m/d

recharge_array_2 = np.zeros((nrow, ncol), dtype=np.float32)

for c in range(n_cols_rech_2):  # Applying recharge to every row to the columns specified.
    for r in range(nrow): 
        recharge_array_2[r, c] = recharge_flux_2

## Boundary conditions

General head boundary on the right-hand side.
No boundary on the left-hand side so use default condition which is a no-flow boundary.

In [49]:
stageleft  = float(sHead)
stageright = float(sHead)

bound_sp1  = []
for il in range(nlay):

    condright = (hk_aquifer * (stageright - zbot) * delc)/10

    for ir in range(nrow):
        bound_sp1.append([il, ir, ncol - 1, stageright, condright])

print('Adding ', len(bound_sp1), 'GHBs for stress period 1.')

ghb_spd = {0: bound_sp1}    

Adding  2400 GHBs for stress period 1.


Specifying starting heads

In [50]:
if nlay == 1:
    ibound = np.ones((nrow, ncol), dtype=np.int32)
    strt = sHead * np.ones((nrow, ncol), dtype=np.float32)

else:
    ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
    strt = sHead * np.ones((nlay, nrow, ncol), dtype=np.float32) 
   

## Stress periods

1 Stress period: steady state only 

In [51]:
nper   = 1 # Number of model stress periods.
perlen = [length_simulation] 

nstp   = [1] # Number of time steps in each stress period.
tsmult = [1]
steady = [True] # True = steady state.

print("One steady-state stress periods set up")

One steady-state stress periods set up


## Input files and running flow model

Running multiple model scenarios:
         
* 1st:   Different recharge scenarios.
     
* 2nd:   Different case studies, representing the different hydraulic conductivities (barrier configurations).


In [52]:
print("The total number of scenarios is: %s" %(
                        number_of_recharge_scenarios*number_of_casestudies))

The total number of scenarios is: 6


## Function to set up and run the flow model

Need to be careful whether I am adding 1 to recharge scenario and case study or not, I have changed things around a bit. 

In [53]:
print(modelName_mt3d)

TB_01_mt3d


In [54]:
def flow_model(rechargeScenario, caseStudy):
    
    modelname = str(modelName) + "_R" + str(rechargeScenario) + "_CS" + str(caseStudy)
    print("Flow model name: " + str(modelname))
        
    modelname_mt3d = modelName_mt3d + "_R" + str(rechargeScenario) + "_CS" + str(caseStudy)
    print("Transport model name: " + str(modelname_mt3d))
    
    mf = flopy.modflow.Modflow(modelname, exe_name=MODFLOW_folder, version='mfnwt') # MODFLOW-NWT
    
    dis = flopy.modflow.ModflowDis(mf, nlay, nrow, ncol, delr=delr, 
                                        delc=delc,top=ztop, botm=botm[1:],
                                        tsmult = tsmult, nper=nper, perlen=perlen, 
                                        nstp=nstp, steady=steady)
        
    print("Discretisation module set up")
       
    # Which hydraulic conductivity array do I use, depending on which scenario is applied:
        
    if caseStudy == 1:
        hk_array = hk_array_1
        print("Setting up hk array for Case Study 1")
            
    elif caseStudy == 2:
        hk_array = hk_array_2
        print("Setting up hk array for Case Study 2")
                   
    elif caseStudy == 3:
        hk_array = hk_array_3
        print("Setting up hk array for Case Study 3")
        
    else:
        print("This case study is not a viable option, choose a value from 1 to 3")
    
    uwp = flopy.modflow.mfupw.ModflowUpw(mf, hk=hk_array, vka=vka, sy=sy, ss=ss, ipakcb=53, iphdry = 53,
                                                 laytyp=laytyp) # MODFLOW- NWT
    print("Upstream weighting package set up")
        
    # --- Solver --- #
            
    nwt = flopy.modflow.mfnwt.ModflowNwt(mf, headtol=1e-05, maxiterout=200, 
                        thickfact=1e-07, linmeth=2, iprnwt=1, ibotav=1, options='SPECIFIED', 
                        Continue=True, dbdtheta=0.9, dbdkappa=1e-05, dbdgamma=0.0, momfact=0.1, 
                        backflag=1, maxbackiter=30, backtol=1.05, backreduce=0.9, maxitinner=50, 
                        ilumethod=2, levfill=5, stoptol=1e-10, msdr=15, iacl=2, norder=1, level=3, 
                        north=7, iredsys=1, rrctols=0.0, idroptol=1, epsrn=0.0001, 
                        hclosexmd=0.0001, mxiterxmd=200) # MODFLOW- NWT fluxtol=1e-3, 
                                                   
    print("Newton solver set up")
        
    # --- Recharge --- #
        
    if rechargeScenario == 1:
        recharge_array = recharge_array_1
        print("Setting up recharge array for Recharge Scenario 1")
    
    elif rechargeScenario == 2:
        recharge_array = recharge_array_2
        print("Setting up recharge array for Recharge Scenario 2")

    else:
        print("This Recharge Scenario is not a viable option, choose a value from 1 to 2")
    
    rch = flopy.modflow.ModflowRch(mf, rech=recharge_array)
        
    print("Recharge package set up")
        
    # --- General head boundaries --- #
        
    ghb = flopy.modflow.ModflowGhb(mf, stress_period_data=ghb_spd)
        
    print("General head boundary package set up")
        
    # --- SETTING UP THE OUTPUT CONTROL --- #
        
    spd = {}
    for strsp in range(nper):
        tmstp = nstp[strsp]
        for t in range(tmstp):
            spd[strsp, t] = ['save budget', 'print budget', 'save head'] 
                                       
    oc = flopy.modflow.ModflowOc(mf, stress_period_data=spd, compact=True)
        
    print("Output control package set up")
        
    # --- SETTING UP BASIC PACKAGE, IBOUND --- #
        
    bas = flopy.modflow.ModflowBas(mf, ibound=1., strt=float(sHead), 
                                       ifrefm=True, ixsec=False, hnoflo=-999.99)
        
    print("Basic package set up") 
        
    # --- LINKING FILE --- #
        
    # Set up the linking file for output from flow model to be used in transport model
    lmt = flopy.modflow.ModflowLmt(mf, output_file_name= (modelname + str('_mt3dLink.ftl')),
                                       output_file_header='extended',
                                       output_file_format='unformatted')   
                                        
    print("Link-Mass Transport package set up") 
        
    # --- NOW I NEED TO COMPILE AND WRITE ALL OF THE INPUT FILES --- #
          
    mf.write_input() # Write the model input files
           
    print("Input written")
        
    # Setting up subprocess AND RUNNING THE MODEL
    #---------------------------------------------------------------
    p = subprocess.Popen([MODFLOW_folder, modelname], shell = False)   
    p.wait()
    #---------------------------------------------------------------
    print("END: Flow model has been run")
    
    return modelname, modelname_mt3d, mf
     

## Running the flow model

In [55]:
modelname, modelname_mt3d, mf = flow_model(1, 1)

Flow model name: TB_01_R1_CS1
Transport model name: TB_01_mt3d_R1_CS1
Discretisation module set up
Setting up hk array for Case Study 1
Upstream weighting package set up
Newton solver set up
Setting up recharge array for Recharge Scenario 1
Recharge package set up
General head boundary package set up
Output control package set up
Basic package set up
Link-Mass Transport package set up
Input written
END: Flow model has been run


# Function to set up and run transport file

In [56]:
print("Modflow model details: %s" %mf)
print("Modflow model name is: %s" %modelname)
print("Mt3d model name is: %s"%modelname_mt3d)
print("Model ws is: %s" %this_model_folder)
print("MT3D executable is: %s" %MT3D_USGS_folder)

Modflow model details: MODFLOW 12 layer(s) 200 row(s) 400 column(s) 1 stress period(s)
Modflow model name is: TB_01_R1_CS1
Mt3d model name is: TB_01_mt3d_R1_CS1
Model ws is: C:\Users\mar886\WaterTableProject\TracersBarriers\tracers_barriers
MT3D executable is: mt3d-usgs_1.1.0_64.exe


In [57]:
def transport_model(modelname, modelname_mt3d, mf):   

    mt = mt3d.Mt3dms(modflowmodel=mf, modelname=modelname_mt3d, model_ws=this_model_folder, 
                         version=mt3d_version, namefile_ext='mtnam', exe_name=MT3D_USGS_folder,  
                         ftlfilename=(modelname + str('_mt3dLink.ftl'))) 
               
    #------------------------------------------------------------------------------
    # BASIC TRANSPORT PACKAGE
        
    ncomp = 1         # Total number of chemical species in simulation.  
    mcomp = 1         # Total number of "mobile" species. 
    sconc= np.zeros((nlay, nrow, ncol), dtype=np.float32)  # initial conc. = 0 yrs   
    nprs = 0          # Flag indicating frequency of output. = 0: results not saved except 
                      # at end of simulation; > 0: saved at times specified by timprs; < 0: saved
                      # whenever the  number of transport steps is an even multiple of nprs.
        
    btn = mt3d.Mt3dBtn(mt, nlay=nlay, nrow=nrow, ncol=ncol, nper=nper, perlen=perlen, 
                           ncomp=ncomp, mcomp=mcomp, sconc=sconc, prsity=prsity,
                           delr=delr, delc=delc, icbund=1, ifmtcn=-1, savucn=True,
                           nprs=nprs, nprobs=1, cinact=0, ssflag='SState', laycon=1) 
        
    #------------------------------------------------------------------------------
    # ADVECTION PACKAGE
        
    adv = mt3d.Mt3dAdv(mt, mixelm=0)  
        
    # mixelm = 0 means that I am using the standard finite-difference method with upstream or 
    # central-in-stream weighting, depending on the value of NADVFD. 
    # Other options, MOC (1); MMOC (2); HMOC (3); TVD (-1). 
        
    #------------------------------------------------------------------------------
    # GENERALISED CONJUGATE GRADIENT SOLVER for MT3D-USGS
        
    gcg = mt3d.Mt3dGcg(mt, mxiter=30, iter1=50, isolve=2, accl=1, cclose=1e-06)
        
    #------------------------------------------------------------------------------
    # REACTION PACKAGE
        
    rc1 = np.zeros((nlay, nrow, ncol), dtype=np.float32)
    rc1[:, :, :] = -1/365.25
        
    isothm = 0      # 0 = no sorption.
    ireact = 100    # 100 = zeroth-order reaction option.
    rc1 = rc1       # First order reaction rate for diffolved phase of first species.
        
    rct= mt3d.Mt3dRct(mt, isothm=isothm, ireact=ireact, igetsc=0, rc1=rc1) 
        
    #------------------------------------------------------------------------------
    # SOURCE-SINK MIXING PACKAGE
        
    crch = np.zeros((nrow, ncol), dtype=np.float32) # The age of recharging water is 0
        
    itype = mt3d.Mt3dSsm.itype_dict()
       
    ssm = mt3d.Mt3dSsm(mt, crch=crch) #, stress_period_data=ssm_data) 
        
    #------------------------------------------------------------------------------
    # DISPERSION PACKAGE 
    al = 1.5 # The longitudinal dispersivity, default = 0.01

#   trpt = 0.1  #ratio of horizontal transverse: longitudinal dispersivity, default = 0.1
#   trpv = 0.01 #ratio of vertical transverse dispersivity: longitudinal dispersivity, default = 0.01
    dmcoef = 1E-4*tortuosity_factor # Effective molecular diffusion coefficient (for water in my model), default = 1E-9 m2/s
        
    # Instantiate up the Dispersion package for MT3D-USGS
    dsp = mt3d.Mt3dDsp(mt, al=al, dmcoef=dmcoef) 
        
    #----- WRITING transport MODEL ------------------------------------------------
         
    mt.write_input()
       
    # - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
    # -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
    # "Manual" changes to the input files
    conc_filename_dissolved = str(modelname_mt3d) + ".ucn"
    conc_filename_sorbed = str(modelname_mt3d) + "_S.ucn"
    mass_filename = str(modelname_mt3d) + ".mas"
    cnf_filename = str(modelname_mt3d) + ".cnf"
        
    ##Add a line to the MT3D .mtnam file re-naming output UCN file + MAS,CNF
    mt_name_file = modelname_mt3d + str('.mtnam')
        
    namfile = open(mt_name_file, 'a')
    namfile.write('data(binary) 201 %s \n' %(conc_filename_dissolved))
    namfile.close()
        
    namfile = open(mt_name_file, 'a')
    namfile.write('data(binary) 301 %s \n' %(conc_filename_sorbed))
    namfile.close()
              
    namfile = open(mt_name_file, 'a')
    namfile.write('data 601 %s \n' %(mass_filename))
    namfile.close()
        
    namfile = open(mt_name_file, 'a')
    namfile.write('data 17 %s \n' %(cnf_filename))
    namfile.close()
                
    ###For USGS need to add DRYCELL keyword to the .btn file on line 3
    mt_btn_file = modelname_mt3d + str('.btn')
        

    btnfile = open(mt_btn_file, "r")
    contents = btnfile.readlines()
    btnfile.close()
    contents.insert(2, "DRYCELL \n")
    btnfile = open(mt_btn_file, "w")
    contents = "".join(contents)
    btnfile.write(contents)
    btnfile.close()
        
    # ADD SSTATE FLAG manually in correct spot

    fpath = os.path.join(this_model_folder, mt_btn_file)
    with open(mt_btn_file, 'r+') as f:
        lines = f.readlines()
        f.seek(0)
        f.truncate()

        a = '%.1E' % Decimal(length_simulation)
        if length_simulation == 1:
            b = "1         1         1 S"
            c = "1         1         1         SState"
        else:
            b = str(a) + "         1         1 S"
            c = str(a) + "         1         1         SState"

        ##

        for line in lines:
            if b in line:
                            line = line.replace(b, c)
            f.write(line)
            
    # Running the transport model
    #--------------------------------
    success, buff = mt.run_model()
    print("Transport model has been run")
    #-------------------------------- 
        
    return mt

## Running the transport model

In [58]:
print(mf)
print(modelname)
print(modelname_mt3d)

MODFLOW 12 layer(s) 200 row(s) 400 column(s) 1 stress period(s)
TB_01_R1_CS1
TB_01_mt3d_R1_CS1


In [59]:
mt = transport_model(modelname, modelname_mt3d, mf)

FloPy is using the following executable to run the model: .\mt3d-usgs_1.1.0_64.exe

 MT3D-USGS - Modular 3D Multi-Species Transport Model [Ver 1.1.0]   
 and based on MT3DMS. MT3D-USGS developed in cooperation by 
 S.S. Papadopulos & Associates and the U.S. Geological Survey

 Using NAME File: TB_01_mt3d_R1_CS1.mtnam

 STRESS PERIOD NO.    1

 TIME STEP NO.    1
 FROM TIME =   0.0000     TO    1.0000    

 Transport Step:    1   Step Size:   1.000     Total Elapsed Time:   1.0000    
 Outer Iter.  1  Inner Iter.  1:  Max. DC =  0.3690E+05  [K,I,J]   12  200    1
 Outer Iter.  1  Inner Iter.  2:  Max. DC =  0.2050E+05  [K,I,J]   12  200    1
 Outer Iter.  1  Inner Iter.  3:  Max. DC =  0.4248E+05  [K,I,J]   12  200    1
 Outer Iter.  1  Inner Iter.  4:  Max. DC =  0.1623E+05  [K,I,J]   12  199   33
 Outer Iter.  1  Inner Iter.  5:  Max. DC =   7840.      [K,I,J]   12  199   57
 Outer Iter.  1  Inner Iter.  6:  Max. DC =  0.4925E+05  [K,I,J]   12  199   85
 Outer Iter.  1  Inner Iter.  7

 Outer Iter.  6  Inner Iter.  1:  Max. DC =  0.1431E-05  [K,I,J]   12  184  398
 Outer Iter.  6  Inner Iter.  2:  Max. DC =  0.3576E-06  [K,I,J]   12   78  387
 Outer Iter.  7  Inner Iter.  1:  Max. DC =  0.1073E-05  [K,I,J]   12  177  339
 Outer Iter.  7  Inner Iter.  2:  Max. DC =  0.2980E-06  [K,I,J]   12   19  202
 Outer Iter.  8  Inner Iter.  1:  Max. DC =  0.1311E-05  [K,I,J]   12   67  178
 Outer Iter.  8  Inner Iter.  2:  Max. DC =  0.3576E-06  [K,I,J]   12   29  343
 Outer Iter.  9  Inner Iter.  1:  Max. DC =  0.1192E-05  [K,I,J]   12   52  345
 Outer Iter.  9  Inner Iter.  2:  Max. DC =  0.3576E-06  [K,I,J]   12  160  385
 Outer Iter. 10  Inner Iter.  1:  Max. DC =  0.1132E-05  [K,I,J]   12  129  361
 Outer Iter. 10  Inner Iter.  2:  Max. DC =  0.2384E-06  [K,I,J]   12   25  221
 Outer Iter. 11  Inner Iter.  1:  Max. DC =  0.1013E-05  [K,I,J]   12  120  377
 Outer Iter. 11  Inner Iter.  2:  Max. DC =  0.2980E-06  [K,I,J]   12    2  389
 Outer Iter. 12  Inner Iter.  1:  Max. D